# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
import numpy as np
import sqlalchemy

#NLTK libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#SK Learn Libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.pipeline import Pipeline,  FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import HashingVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database and add to pandas dataframe
def loadData():
    engine = sqlalchemy.create_engine('sqlite:///pipeline.db')
    df = pd.read_sql_table('messages', engine)  
    x = df.message.values  # Messge column only
    y = df.drop(['id', 'message', 'original', 'genre'], axis = 1) #drop the following columns from df: 'id', 'message', 'original', 'genre'

    return x, y

### 2. Write a tokenization function to process your text data

In [3]:
# function to prepare message text and tokenization
def tokenize(text):
    text = text.lower() # make text lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # remove everything but text and numbers
    text = word_tokenize(text) # tokenize words to break them into individual words
    text = [t for t in text if t not in stopwords.words("english")] # remove stops words sycg as 'a, an, in'
    text = [PorterStemmer().stem(t) for t in text] # stem and lemmetize words to bring them to the root form of the word
    text = [WordNetLemmatizer().lemmatize(t, pos='v') for t in text]
    
    return text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# pipeline No.1
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), # Creates a matrix that counts the words
    ('tfidf', TfidfTransformer()), # divides the above count per message.
    ('clf',  MultiOutputClassifier(RandomForestClassifier())) # runs a classifier that uses decision tree to attempt classify the messages
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#training the data using the pipeline


def main(pipeline, x, y, CVparameters={}, GSCV=False):
    # split the data into test & train datasets
    X_train, X_test, y_train, y_test = train_test_split(x, y)
    
    if GSCV == True:
        # create grid search
        cv = GridSearchCV(pipeline, param_grid=CVparameters)
        # fit the CV optimized training data to the pipeline classifier
        cv.fit(X_train, y_train)
        # Use the x (message) to predict the message category, based on the traning data (CV optimized)
        y_pred = cv.predict(X_test)
    else:
        # fit the training data to the pipeline classifier
        pipeline.fit(X_train, y_train)
        # Use the x (message) to predict the message category, based on the traning data
        y_pred = pipeline.predict(X_test)

    # Create aan empty list and load the columns names of y into the list
    l = []
    y =loadData()[1]
    for cols in y:
        l.append(cols)

    # loop & enumerate through each item in 'y' so that is runs the classification report on each class - testing the accuracy
    for i, col in enumerate(l):
        print("------" + col)
        col_y_test = list(y_test.values[:,i])
        col_y_pred = list(y_pred[:, i])
        classRep = classification_report(col_y_test, col_y_pred)
        print(classRep)
        
    return y_pred
    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#Load the data to run
x, y = loadData()

#run the main function, which also prints the classification report to check for performance/accuracy
main(pipeline1, x, y)




------related
             precision    recall  f1-score   support

          0       0.62      0.47      0.54      1504
          1       0.85      0.91      0.88      5002
          2       0.34      0.50      0.40        48

avg / total       0.79      0.81      0.80      6554

------request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5413
          1       0.81      0.44      0.57      1141

avg / total       0.88      0.88      0.87      6554

------offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

------aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3819
          1       0.75      0.59      0.66      2735

avg / total       0.75      0.75      0.74      6554

------medical_help
          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6270
          1       0.11      0.00      0.01       284

avg / total       0.92      0.96      0.94      6554

------weather_related
             precision    recall  f1-score   support

          0       0.87      0.95      0.91      4733
          1       0.83      0.63      0.71      1821

avg / total       0.86      0.86      0.85      6554

------floods
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      6017
          1       0.87      0.45      0.60       537

avg / total       0.95      0.95      0.94      6554

------storm
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      5925
          1       0.75      0.41      0.54       629

avg / total       0.92      0.93      0.92      6554

------fire
             precision    recall  f1-score   support

          0       0.99      

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
#from sklearn.model_selection import GridSearchCV

parameters ={
    'tfidf__norm' : ['l1', 'l2']
    }

    #'clf__n_estimators': [100]
#cv = GridSearchCV(pipeline, param_grid=parameters)
#cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
#run the main function, which also prints the classification report to check for performance/accuracy
main(pipeline1, x, y, parameters, True)


------related
             precision    recall  f1-score   support

          0       0.62      0.47      0.53      1542
          1       0.84      0.91      0.88      4965
          2       0.35      0.38      0.37        47

avg / total       0.79      0.80      0.79      6554

------request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5389
          1       0.81      0.40      0.53      1165

avg / total       0.87      0.88      0.86      6554

------offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6519
          1       0.00      0.00      0.00        35

avg / total       0.99      0.99      0.99      6554

------aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.79      3813
          1       0.74      0.60      0.66      2741

avg / total       0.74      0.74      0.74      6554

------medical_help
          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.87      0.95      0.91      4764
          1       0.83      0.61      0.71      1790

avg / total       0.86      0.86      0.85      6554

------floods
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      6019
          1       0.87      0.38      0.53       535

avg / total       0.94      0.94      0.93      6554

------storm
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      5944
          1       0.76      0.41      0.53       610

avg / total       0.93      0.93      0.92      6554

------fire
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6484
          1       0.50      0.01      0.03        70

avg / total       0.98      0.99      0.98      6554

------earthquake
             precision    recall  f1-score   support

          0       0.97      0.99 

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
# second pipeline2 - which includes a feature union with hashing vectorizer and ada boost classifer
#in order to improvde the model.
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('hasVect', HashingVectorizer())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])


main(pipeline2, x, y, parameters, False)

------related
             precision    recall  f1-score   support

          0       0.52      0.13      0.21      1537
          1       0.78      0.96      0.86      4966
          2       0.33      0.16      0.21        51

avg / total       0.71      0.76      0.70      6554

------request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5439
          1       0.76      0.52      0.62      1115

avg / total       0.88      0.89      0.88      6554

------offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      0.99      0.99      6554

------aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.80      3825
          1       0.77      0.58      0.66      2729

avg / total       0.75      0.75      0.75      6554

------medical_help
          

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

### 9. Export your model as a pickle file

In [10]:
import pickle
pickle.dump(model.best_estimator_, open(r'/classifier.pkl', 'wb'))

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.